# Accesso programático a bases de datos de NCBI

La mayoría de las bases de datos NCBI comparten un único sistema de búsqueda y recuperación de datos, llamado Entrez/E-utils.

Entrez es el sistema que maneja a las bases de datos.

E-Utils son aplicaciones/interfaces que permiten trabajar con entrez.

## Entrez Programming Utilities (E-utilities) 

Es un conjunto de nueve programas en servidores web que parmiten acceder a sistema Entrez:

Se accede a cada uno de estos programas con una URL particular y permite hacer consultas mediante una REST-API por HTTP. Las respuestas con generalmente en formato XML, pero puede ser fasta u otro formato especial para algunos pedidos.

Los programas son:

### EInfo (database statistics)

url: eutils.ncbi.nlm.nih.gov/entrez/eutils/einfo.fcgi

Provee datos de cada base de datos, como ser la fecha de la última actualización, cantidad de registros, los campos de cada registros.

### ESearch (text searches)

url: eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi

Permite hacer búsqueda por texto. Devuelte una lista de identificadores de los registros seleccionados para ser luego usados en otros servicios como ESummary, EFetch o Elink.

### ESummary (document summary downloads)

url: eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi

Devuelve resúmenes de cada registro a partir de una lista de identificadores.

### EFetch (data record downloads)

url: eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi

Devuelve registros completos a partir de una lista de identificadores.

### ELink (Entrez links)

url: eutils.ncbi.nlm.nih.gov/entrez/eutils/elink.fcgi

Acepta una lista de identificadores y devuelve una lista de identificadores de registros relacionadoes de la misma base de datos u otra base de datos.

### EPost (UID uploads)

url: eutils.ncbi.nlm.nih.gov/entrez/eutils/epost.fcgi

Acepta una lista de identificadores que se almacenan en un History Server, lo que permite acceder a ellos más tarde.

### EGQuery (global query)

url: eutils.ncbi.nlm.nih.gov/entrez/eutils/egquery.fcgi

Hace una búsqueda de texto global en todas las bases de datos.

### ESpell (spelling suggestions)

url: eutils.ncbi.nlm.nih.gov/entrez/eutils/espell.fcgi

Recupera sugerencias de escritura para entradas de texto.

### ECitMatch (batch citation searching in PubMed)

url: eutils.ncbi.nlm.nih.gov/entrez/eutils/ecitmatch.cgi

Recupera PMCIDs para una listas de citas.

Nosotros vamos a centrarnos en algunos de estos programas:

- EInfo
- ESearch
- ESummary
- EFetch

In [ ]:
# Cargamos la librerías necesarias
library(httr)
library(xml2)
library(rentrez)

## EInfo - Estadísticas de las bases de datos

### De la forma complicada

In [ ]:
# Defino la url del programa
einfo_url <- "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/einfo.fcgi"

# Hago el pedido http y obtengo la respuesta
response <- GET(einfo_url)

In [ ]:
# Me aseguro que la respuesta es exitosa.
# El status code debe ser 200
status_code(response)

In [ ]:
# Convierto la respuesta a una lista de R
contenido <- as_list(content(response))
head(contenido)

In [ ]:
# La función names me ayuda a ver la estructura de la lista.

# names(contenido)

# contenido$eInfoResult

# contenido$eInfoResult$DbList

nombres_db <- unlist(contenido$eInfoResult$DbList)
nombres_db


In [ ]:
?GET

In [ ]:
# Ahora veamos una base de datos con más detalles
mi_db <- 'pubmed'

# EInfo acepta un argumento 'db' para pedir detalles de una base de datos en particular.
response <- GET(einfo_url, query=list(db=mi_db))

# Verifico el status_code
status_code(response)

In [ ]:
contenido <- as_list(content(response))
contenido

In [ ]:
# ¿Qué datos devuelve?
names(contenido)
names(contenido$eInfoResult)
names(contenido$eInfoResult$DbInfo)

In [ ]:
# Recuperamos el nombre de la db
contenido$eInfoResult$DbInfo$DbName

In [ ]:
# Recuperamos la descripción
contenido$eInfoResult$DbInfo$Description

In [ ]:
# Recuperamos la cantidad de registros
contenido$eInfoResult$DbInfo$Count

In [ ]:
# Recuperamos la lista de campos de los registros
contenido$eInfoResult$DbInfo$FieldList

In [ ]:
# Armamos un dataframe con la descripción de cada campo
tabla <- sapply(
    contenido$eInfoResult$DbInfo$FieldList,
    function(x) {return(unlist(x[c("Name", "Description")]))}
)

tabla <- t(tabla)
head(tabla)

In [ ]:
# Probemos con otras bases de datos
# Armar una tabla similar a la anterior, con los campos Name, Description y TermCount

### Usar EInfo con el paquete rentrez

Ahora que hicimos un ejemplo de la forma complicada,
veamos como podemos hacer esto más sencillo usando la
librería de R **rentrez**.

In [ ]:
# Cargamos el la librería de R que 
library(rentrez)

In [ ]:
# Recupero la lista de todas las bases de datos de NCBI
ncbi_dbs <- entrez_dbs()
ncbi_dbs

In [ ]:
# Muestra un resumen de la base de datos
entrez_db_summary("pubmed")

In [ ]:
# Muestra un resumen de la base de datos
entrez_db_summary("pubmed")

In [ ]:
# Muestra los campos de búsqueda disponible en una base de datos
campos <- entrez_db_searchable("pubmed")

In [ ]:
campos_mat <- sapply(campos, function(x) {return(unlist(x[c('Name','FullName')]))})
campos_df <- as.data.frame(t(campos_mat))
head(campos_df)

## ESearch - Búsquedas de texto

Para usar ESearch tenemos que:

- Definir una base de datos.
- Establecer el texto de la búsqueda.
- Definir el número máximo de identificadores a recuperar.

Vamos a usar la función 'entrez_search' de la librería 'rentrez'.

In [ ]:
?entrez_search

#### ¿Cómo escribir el texto de búsqueda?

Cuando vimos los ejemplos de EInfo, vimos que cada base de datos tiene definido una serie de campos de búsqueda.
Para hacer las búsquedas tenemos que definir los términos que queremos buscar y en qué campos queremos hacer la búsqueda.

Por ejemplo, si queremos buscar artículos del autor Huntington, pero no queremos que nos devuelvan artículos sobre la enfermedad de Huntington, tenemos que poder especificar que el término de búsqueda "Huntington" se realice únicamente sobre el campo de los autores.

El formato básico para definir las búsquedas es este:

    texto_de_busqueda[campo]
    # El campo puede ser el Name o el FullName
    # por ejemplo:
    huntington[AUTH]
    huntington[Author]


In [ ]:
# Probemos el ejemplo
mi_db <- "pubmed"
mi_term <- "huntington[Author]"
max_artic <- 10

# Con la función entrez_search 
resultado <- entrez_search(db=mi_db, term=mi_term, retmax=max_artic)

In [ ]:
# Vemos el contenido del resultado
names(resultado)

# Nos muestra la cantidad de resultados de la búsqueda
resultado$count

# Nos devuelve los IDS de los 10 que le pedimos
resultado$ids

Podemos hacer búsquedas más complejos combinando búsquedas en más campos con operadores lógicos: AND, OR, NOT

    texto_de_busqueda_1[campo_1] OP texto_de_busqueda_2[campo_2] OP texto_de_busqueda_3[campo_3] ...
    # El campo puede ser el Name o el FullName
    # por ejemplo:
    huntington[Author] AND review[Publication Type]

In [ ]:
# Probemos el ejemplo
mi_db <- "pubmed"
mi_term <- "huntington[Author] AND review[Publication Type]"
max_artic <- 10

# Con la función entrez_search 
resultado <- entrez_search(db=mi_db, term=mi_term, retmax=max_artic)

In [ ]:
resultado$count

In [ ]:
# Probemos el ejemplo
mi_db <- "pubmed"
mi_term <- "huntington[Author] AND review[Publication Type] AND (2000[Publication Date] OR 2010[Publication Date])"
max_artic <- 10

# Con la función entrez_search 
resultado <- entrez_search(db=mi_db, term=mi_term, retmax=max_artic)

In [ ]:
resultado$ids

## ESummary - Resumenes de registros

Para usar ESummary tenemos que:

- Definir una base de datos.
- Pasarle un identificador o una lista separados por comas (no más de 100)

Usamos la función 'entrez_summary' de 'rentrez'.

Continuemos con el ejemplo que venimos trabajando y recuperemos el resumen de los registros que seleccionamos en la búsqueda anterior.

In [ ]:
summary <- entrez_summary(db='pubmed', id=resultado$ids)

In [ ]:
names(summary)
summary$'21117518'

In [ ]:
# Armemos un dataframe con el id, el título, 
# el nombre del primer autor y del ultimo autor, y el nombre de la revista.

selected_fields <- c('uid', 'title', 'lastauthor', 'sortfirstauthor', 'source')
data <- sapply(summary, function(x) unlist(x[selected_fields]) )
data <- as.data.frame(t(data))
data

## EFetch - Recuperar registros completos

Para usar EFetch tenemos que:

- Definir una base de datos.
- Pasarle un identificador o una lista separados por comas (no más de 100)
- El formato de los datos a recuperar.

Usamos la función 'entrez_fetch' de 'rentrez'.

Continuemos con el ejemplo que venimos trabajando y recuperemos el resumen de los registros que seleccionamos en la búsqueda anterior.

In [ ]:
records <- entrez_fetch(db='pubmed', id=resultado$ids, rettype='xml')

In [ ]:
library(xml2)
doc <- as_list(read_xml(records))
names(doc)

names(doc$PubmedArticleSet)

names(doc$PubmedArticleSet[[1]])

names(doc$PubmedArticleSet[[1]]$MedlineCitation)

doc$PubmedArticleSet[[1]]$MedlineCitation$PMID

doc$PubmedArticleSet[[1]]$MedlineCitation$Article$Abstract

In [ ]:
# Recupero todos los abstracts
abstracts <- sapply(doc$PubmedArticleSet, function(x) unlist(x$MedlineCitation$Article$Abstract))
abstracts

In [ ]:
# Si solo queremos los abstracts podemos cambiar el tipo de 
# datos que queremos recuperar en la función entrez_fetch
# Aunque es más dificil manipular los resultados después.
# https://www.ncbi.nlm.nih.gov/books/NBK25499/table/chapter4.T._valid_values_of__retmode_and/?report=objectonly

records <- entrez_fetch(db='pubmed', id=resultado$ids, rettype='abstract')

In [ ]:
cat(records)

## Ejemplos con otras bases de datos

### Ejemplo 1 - Busquemos secuencias de genomas completos de *E. coli*



In [ ]:
entrez_db_searchable(db='nuccore')

In [ ]:
# Primer paso, consulta con ESearch
mi_db <- "nuccore"
mi_term <- "escherichia coli[ORGANISM] AND plasmid[Title]"
max_artic <- 100

resultados <- entrez_search(db=mi_db, term=mi_term, retmax=max_artic)

In [ ]:
# Recupero el resumen
records <- entrez_summary(
    db='nuccore',
    id=resultados$ids
)

In [ ]:
plasmid_size <- t(sapply(records, function(x) unlist(x[c("genome", "slen")])))
hist(log(as.numeric(plasmid_size[,"slen"]), 10), 20)

In [ ]:
resultados$ids[1]

In [ ]:
# Recupero la secuencia en formato fasta
records <- entrez_fetch(
    db='nuccore',
    id=resultados$ids[1],
    rettype='fasta'
)

In [ ]:
substr(records[[1]], 1, 500)

In [ ]:
# Recupero la secuencia en formato gb, anotaciones
records <- entrez_fetch(
    db='nuccore',
    id=resultados$ids[2],
    rettype='gb'
)

In [ ]:
cat(records)

In [ ]:
# Puedo convertir este formato a un formato de texto
# if (!requireNamespace("BiocManager", quietly = TRUE))
#     install.packages("BiocManager")
# BiocManager::install("genbankr", force=TRUE)
# install.packages("stringi")
# library("stringi")


In [ ]:
# Puedo convertir este formato a un formato de texto
library(genbankr)
library(GenomicRanges)
gb <- readGenBank(text = records)

In [ ]:
gb

In [ ]:
genes(gb)

In [ ]:
transcripts(gb)

In [ ]:
entrez_db_searchable(db="structure")

In [ ]:
term <- "nucleoprotein[Protein Name] AND 0:3.0[Resolution] AND complex_protein[filt] AND arenaviridae[ORGN]"
resultados <- entrez_search(
    db='structure',
    term=term
)

In [ ]:
resumen <- entrez_summary(id=resultados$ids, db='structure', ret_type="xml")

In [ ]:
names(resumen$'152393')

In [ ]:
structures <- sapply(resumen, function(x) unlist(x[c('pdbacc', 'pdbdescr', 'pdbclass', 'proteinmoleculecount')]))
structures <- t(structures)
structures

In [ ]:
record <- entrez_link(
    dbfrom='structure',
    db='protein',
    id=resultados$ids[1]
)

In [ ]:
record$links

In [ ]:
sequences <- entrez_fetch(
    db='protein',
    id=record$links$structure_protein,
    rettype='fasta'
)

In [ ]:
cat(sequences)